In [1]:
%cd C:\mimic-iv-2.2\mimic-iv-2.2\hosp

C:\mimic-iv-2.2\mimic-iv-2.2\hosp


In [2]:
import pandas as pd

In [3]:
df_d_icd_diagnoses = pd.read_csv('d_icd_diagnoses.csv')
df_diagnoses_icd = pd.read_csv("diagnoses_icd.csv")


In [ ]:
# 1.전체에서 암icd코드 있는 행들의 subject_id를 중복제거하고 추출 ==>암있는 환자들만 나옴 
# 2. 암있는 환자들중 icd_code가 고혈압인 행들만 골라서 그 subject_id추출한다.==> 암있고 고혈압인 subject_id만 추출가능.
# 3. .seq_num==1 인 애들중  코드가 암인 애들만 추출
# 4. 다른 테이블들이랑 병합.

In [16]:
import pandas as pd

# 고혈압 진단 코드 정규식
high_bp_regex = r'^I1[0-5]$|^401'

#암 진단 코드 정규식  
#ICD-9 코드 분류 (140-239)
#ICD-10 코드 분류 (C00-C96, D00-D09)
cancer_regex = r'^(D(0[0-9])|C([0-8][0-9]|9[0-6])|(1[4-9][0-9]|2[0-3][0-9])).*'


# 암 진단 코드 추출
cancer_df = df_diagnoses_icd[df_diagnoses_icd['icd_code'].str.match(cancer_regex)]

# 암 진단을 받은 환자 ID 추출
cancer_patient_ids = cancer_df['subject_id'].unique()

# 암 진단을 받은 환자들의 데이터 추출
cancer_diagnoses_merged = pd.merge(df_diagnoses_icd, df_d_icd_diagnoses, on=['icd_code'])
cancer_patients_data = cancer_diagnoses_merged[cancer_diagnoses_merged['subject_id'].isin(cancer_patient_ids)]



# 암 및 고혈압을 동시에 가진 환자들의 데이터 추출 (첫 번째 진단만 포함)
cancer_high_bp_ids = cancer_patients_data[cancer_patients_data['icd_code'].str.match(high_bp_regex)]['subject_id'].unique()
cancer_high_bp_df = cancer_diagnoses_merged[(cancer_diagnoses_merged['subject_id'].isin(cancer_high_bp_ids)) & 
                                            (cancer_diagnoses_merged['seq_num'] == 1) & 
                                            (cancer_diagnoses_merged['icd_code'].str.match(cancer_regex))]

cancer_high_bp_df

,subject_id,hadm_id,seq_num,icd_code,icd_version_x,icd_version_y,long_title
393337,10071129,25200387,1,1890,9,9,"Malignant neoplasm of kidney, except pelvis"
393339,10094971,20200109,1,1890,9,9,"Malignant neoplasm of kidney, except pelvis"
393351,10171405,23985269,1,1890,9,9,"Malignant neoplasm of kidney, except pelvis"
393356,10171405,29677692,1,1890,9,9,"Malignant neoplasm of kidney, except pelvis"
393364,10322775,24742385,1,1890,9,9,"Malignant neoplasm of kidney, except pelvis"
...,...,...,...,...,...,...,...
4859732,19128927,28814179,1,C44692,10,10,Other specified malignant neoplasm of skin of ...
4859827,19282415,28695316,1,C004,10,10,"Malignant neoplasm of lower lip, inner aspect"
4859828,19290501,28187167,1,C8468,10,10,"Anaplastic large cell lymphoma, ALK-positive, ..."
4860174,19851929,29152819,1,1643,9,9,Malignant neoplasm of posterior mediastinum
